/content


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam


import numpy as np
import os
import cv2

def extract_rgb_histogram(image_path, bins_per_channel=8):
    """Extracts a flattened RGB histogram from an image."""
    image = cv2.imread(image_path)
    if image is None:  # Check if the image was loaded successfully
        print(f"Warning: Unable to load image at {image_path}. Skipping...")
        return None  # Return None to indicate failure to load
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB

    # Compute histograms for each channel
    histogram = [cv2.calcHist([image], [i], None, [bins_per_channel], [0, 256]) for i in range(3)]
    histogram = np.concatenate(histogram).flatten()

    # Normalize the histogram
    histogram = histogram / np.sum(histogram)
    return histogram

def load_data(dataset_path, bins_per_channel=8):
    X = []
    y = []
    labels = os.listdir(dataset_path)
    label_dict = {label: idx for idx, label in enumerate(labels)}

    expected_histogram_length = bins_per_channel * 3  # Since you have 3 channels (RGB)

    for label in labels:
        class_path = os.path.join(dataset_path, label)
        for image_file in os.listdir(class_path):
            image_path = os.path.join(class_path, image_file)
            histogram = extract_rgb_histogram(image_path, bins_per_channel)
            if histogram is not None:
                if len(histogram) != expected_histogram_length:
                    print(f"Unexpected histogram length for {image_path}. Expected: {expected_histogram_length}, Got: {len(histogram)}")
                    continue  # Skip this histogram
                X.append(histogram)
                y.append(label_dict[label])

    # Ensure all elements in X have the same shape for conversion to a NumPy array
    X = np.array(X, dtype=object)  # Use dtype=object temporarily if needed
    # Verify if all histograms are of equal size, otherwise, investigate the cause
    if not all(len(hist) == expected_histogram_length for hist in X):
        raise ValueError("Not all histograms have the expected length. Please check the data.")

    # Correctly reshape or cast X as needed here, once you've ensured all histograms are consistent
    # Example assuming all histograms are consistent:
    # X = np.stack(X)  # This converts the list of arrays into a single 2D array if they are all of equal size

    y = tf.keras.utils.to_categorical(y)  # Convert labels to one-hot encoding if needed
    return X, y




def create_model(input_dim, num_classes):
    """
    Creates a 3-layer neural network with specified input dimension and number of output classes.

    Parameters:
    - input_dim: Integer, the size of the input layer (number of features).
    - num_classes: Integer, the number of classes for the output layer.

    Returns:
    - model: A Keras Sequential model.
    """
    model = Sequential([
        Dense(50, input_dim=input_dim, activation='relu'),  # First hidden layer
        Dense(100, activation='relu'),                      # Second hidden layer
        Dense(150, activation='relu'),                      # Third hidden layer
        Dense(num_classes, activation='softmax')            # Output layer
    ])
    return model

def compile_model(model):
    """
    Compiles the neural network model with an optimizer, loss function, and evaluation metrics.

    Parameters:
    - model: A Keras Sequential model.

    No return value.
    """
    model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
# Example usage
dataset_path = '/content/drive/MyDrive/ColabNotebooks/SolarpanelImage/'
X, y = load_data(dataset_path)






In [ ]:
# Split the data (consider using sklearn's train_test_split)
from sklearn.model_selection import train_test_split

X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25)


In [ ]:
input_dim = 24  # This is for the 24-dimensional RGB histograms

# Define the number of classes for your output layer
num_classes = 6

# Create the model
model = create_model(input_dim=input_dim, num_classes=num_classes)

# Compile the model
compile_model(model)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

def train_model(model=model, X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val, epochs=500, batch_size=2):
    """
    Trains the model on the training data with early stopping and dynamically decreasing learning rate.

    Parameters:
    - model: A Keras Sequential model.
    - X_train: Feature data for training.
    - y_train: Labels for training.
    - X_val: Feature data for validation.
    - y_val: Labels for validation.
    - epochs: Number of epochs to train the model.
    - batch_size: Batch size for training.

    Returns:
    - history: A history object containing training history metrics.
    """
    # Initialize the EarlyStopping callback
    early_stopping = EarlyStopping(monitor='val_loss',
                                   patience=10,
                                   verbose=1,
                                   restore_best_weights=True)

    # Initialize the ReduceLROnPlateau callback
    reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                                  factor=0.2,  # New learning rate = factor * previous learning rate
                                  patience=5,  # Number of epochs with no improvement after which learning rate will be reduced
                                  verbose=1,
                                  min_lr=1e-6)  # Lower bound on the learning rate

    # Fit the model with the training data and the callbacks
    history = model.fit(X_train, y_train,
                        validation_data=(X_val, y_val),
                        epochs=epochs,
                        batch_size=batch_size,
                        callbacks=[early_stopping, reduce_lr])  # Add both callbacks here
    return history


In [ ]:
input_dim = 24  # This is for the 24-dimensional RGB histograms

# Define the number of classes for your output layer
num_classes = 6

# Create the model
model = create_model(input_dim=input_dim, num_classes=num_classes)

# Compile the model
compile_model(model)

In [ ]:
history = train_model()

Epoch 1/500
271/271 [==============================] - 2s 3ms/step - loss: 1.6281 - accuracy: 0.3457 - val_loss: 1.4818 - val_accuracy: 0.4199 - lr: 0.0010
Epoch 2/500
271/271 [==============================] - 1s 3ms/step - loss: 1.3928 - accuracy: 0.4436 - val_loss: 1.3517 - val_accuracy: 0.4475 - lr: 0.0010
Epoch 3/500
271/271 [==============================] - 1s 2ms/step - loss: 1.3502 - accuracy: 0.4695 - val_loss: 1.3625 - val_accuracy: 0.4254 - lr: 0.0010
Epoch 4/500
271/271 [==============================] - 1s 2ms/step - loss: 1.2860 - accuracy: 0.5046 - val_loss: 1.3033 - val_accuracy: 0.4917 - lr: 0.0010
Epoch 5/500
271/271 [==============================] - 1s 2ms/step - loss: 1.2710 - accuracy: 0.5157 - val_loss: 1.3649 - val_accuracy: 0.4696 - lr: 0.0010
Epoch 6/500
271/271 [==============================] - 1s 4ms/step - loss: 1.2587 - accuracy: 0.5213 - val_loss: 1.3219 - val_accuracy: 0.4862 - lr: 0.0010
Epoch 7/500
271/271 [==============================] - 1s 4ms/st

In [ ]:
X_train = np.array(X_train).astype(np.float32)
X_val = np.array(X_val).astype(np.float32)
X_test = np.array(X_test).astype(np.float32)

In [ ]:
print("X_train shape:", X_train.shape, "dtype:", X_train.dtype)
print("X_val shape:", X_val.shape, "dtype:", X_val.dtype)
print("X_test shape:", X_test.shape, "dtype:", X_test.dtype)
print("y_train shape:", y_train.shape, "dtype:", y_train.dtype)
print("y_val shape:", y_val.shape, "dtype:", y_val.dtype)

X_train shape: (541, 24) dtype: float32
X_val shape: (181, 24) dtype: float32
X_test shape: (181, 24) dtype: float32
y_train shape: (541, 6) dtype: float32
y_val shape: (181, 6) dtype: float32


In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

6/6 [==============================] - 0s 4ms/step - loss: 1.2574 - accuracy: 0.5470
Test Loss: 1.2573987245559692
Test Accuracy: 0.5469613075256348
